[View in Colaboratory](https://colab.research.google.com/github/sebastianjaszczur/sparser-layer/blob/master/Sparser.ipynb)

In [0]:
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 0s 4us/step


In [0]:
x_train.shape

(404, 13)

In [0]:
from keras import Sequential

In [0]:
from keras.layers import Dense, Input, Layer, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.activations import relu
from keras.initializers import RandomNormal 
from keras import backend as K

In [0]:
UID = 1
def uid():
  global UID
  UID += 1
  return UID

In [0]:
# Sparser layer with less weights.
class Sparser(Layer):
    def __init__(self, size, **kwargs):
        self.size = int(size)
        super(Sparser, self).__init__(**kwargs)

    def build(self, input_shape):
        assert(input_shape[1] == self.size)
        super(Sparser, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        def basic_sparse(x):
          if x.shape[1] > 2:
            half = int(x.shape[1] // 2)
            shalf = int(x.shape[1] - half)
            x1 = x[:, :half]
            x2 = x[:, half:]
            
            w = [[self.add_weight(name='w'+str(uid()), shape=(half,),
                                  initializer=RandomNormal(mean=0.0, stddev=0.5, seed=None),
                                  trainable=True)
                  for i in range(2)] for j in range(2)]
            
            y1 = w[0][0] * x1 + w[0][1] * x2
            y2 = w[1][0] * x1 + w[1][1] * x2
            f1 = basic_sparse(y1)
            f2 = basic_sparse(y2)
            return K.concatenate([f1, f2])
          else:
            return x
        return basic_sparse(x)

    def compute_output_shape(self, input_shape):
        return input_shape

In [0]:
def get_model(layer=Dense, lsize=256, lr=0.003):
    inp = Input((13,), dtype="float32")
    prep = Dense(lsize)
    prep.trainable = False
    prep = prep(inp)
    mid = layer(lsize)(prep)
    mid = Activation('relu')(mid)
    mid = layer(lsize)(mid)
    mid = Activation('relu')(mid)
    mid = layer(lsize)(mid)
    mid = Activation('relu')(mid)
    final = Dense(1)
    final.trainable = False
    final = final(mid)

    model = Model(inputs=inp, outputs=final)
    adam = Adam(lr=lr)
    model.compile(optimizer=adam,
                  loss='mse')
    return model

In [0]:
model = get_model(Dense, 128)
model.fit(x_train, y_train, epochs=10, batch_size=4, validation_data=(x_test, y_test))

Train on 404 samples, validate on 102 samples
Epoch 1/10
404/404 [==============================] - 6s 16ms/step - loss: 196.7714 - val_loss: 91.3666
Epoch 2/10
404/404 [==============================] - 1s 2ms/step - loss: 81.9851 - val_loss: 77.1448
Epoch 3/10
404/404 [==============================] - 1s 2ms/step - loss: 71.8224 - val_loss: 61.6950
Epoch 4/10
404/404 [==============================] - 1s 2ms/step - loss: 79.9837 - val_loss: 69.8852
Epoch 5/10
404/404 [==============================] - 1s 2ms/step - loss: 80.3199 - val_loss: 65.7092
Epoch 6/10
404/404 [==============================] - 1s 2ms/step - loss: 65.4707 - val_loss: 72.6224
Epoch 7/10
404/404 [==============================] - 1s 2ms/step - loss: 67.0996 - val_loss: 102.3761
Epoch 8/10
404/404 [==============================] - 1s 2ms/step - loss: 63.9766 - val_loss: 60.5895
Epoch 9/10
404/404 [==============================] - 1s 2ms/step - loss: 68.9157 - val_loss: 68.0684
Epoch 10/10
404/404 [============

In [0]:
model = get_model(Sparser, 128)
model.fit(x_train, y_train, epochs=10, batch_size=4, validation_data=(x_test, y_test))

Train on 404 samples, validate on 102 samples
Epoch 1/10
404/404 [==============================] - 77s 190ms/step - loss: 292.3973 - val_loss: 85.1260
Epoch 2/10
404/404 [==============================] - 35s 86ms/step - loss: 80.4807 - val_loss: 75.9410
Epoch 3/10
404/404 [==============================] - 34s 85ms/step - loss: 68.4536 - val_loss: 68.0781
Epoch 4/10
268/404 [==================>...........] - ETA: 11s - loss: 69.8054

404/404 [==============================] - 34s 85ms/step - loss: 65.4779 - val_loss: 64.9852
Epoch 5/10
404/404 [==============================] - 34s 85ms/step - loss: 63.5468 - val_loss: 64.0440
Epoch 6/10
404/404 [==============================] - 34s 85ms/step - loss: 62.6695 - val_loss: 62.3105
Epoch 7/10
404/404 [==============================] - 34s 85ms/step - loss: 66.2048 - val_loss: 63.8538
Epoch 8/10
108/404 [=======>......................] - ETA: 24s - loss: 57.3301

404/404 [==============================] - 35s 86ms/step - loss: 62.8452 - val_loss: 62.6100
Epoch 9/10
404/404 [==============================] - 34s 85ms/step - loss: 61.5182 - val_loss: 62.5345
Epoch 10/10
404/404 [==============================] - 35s 86ms/step - loss: 61.9458 - val_loss: 62.7524


In [0]:
(sum(int(w.shape[0]) for w in model.weights) - 129*14) / 3


1024.0